# Static 2D Guiding Sonification

**Task**: design a sonification to guide a human operator (e.g. surgeon) to reach a 2D target as precise and fast as possible, i.e. create a sonification using the actual and nominal 2D position so that the target location can be found at least time and with as little distance from start to goal target location. 

This notebook provides a template
* a plot window shows the start position
* mouse events are connected to callback functions to anchor actions 
    * click the mouse to update the sonification
    * move the mouse to attain the target
    * a high pitched sound is played once the target is reached
    * on releasing the Mouse, the actual trajectory is appended to a list of runs
* Three function (son_init, son_update and son_quit) allow you to modify the sonification interactively: you can always interact and listen between modifications
* some rudimentary evaluation methods (RMSE) are defined, but feel free to analyze the interaction data in more detail.

In [1]:
# heading/imports
import time, pickle
from datetime import datetime
from threading import Thread
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
import sc3nb as scn

Using matplotlib backend: Qt5Agg


<IPython.core.display.Javascript object>

In [2]:
sc = scn.startup()  # opt. add arg sclangpath="/path/to/sclang"

Starting sclang...
Done.
Registering UDP callback...
Done.
Booting server...
Done.
"sc3nb started";
-> sc3nb started
sc3> sc3> 


In [3]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [4]:
class StaticTargetTask:

    def __init__(self, num_targets=5, plot_flag=True):
        # generate new targets 
        self.num_targets = num_targets
        self.targets = np.random.uniform(low=0, high=1, size=(self.num_targets, 2))
        self.count = 0
        self.target = self.targets[self.count, :]
        self.plist = []
        self.runs = []
        self.run_flag = False
        self.plot_flag = plot_flag
        self.son_init = None
        self.son_update = None
        self.son_quit = None

        # create plot and bind callback functions
        self.fig = plt.figure(figsize=(5,5))
        self.ax = plt.subplot(111)
        self.p0, = self.ax.plot([0.5], [0.5], "ro", ms=4)
        self.p1, = self.ax.plot([], [], "r-", lw=0.5)
        self.ax.axis([0, 1, 0, 1])
        self.mngr = plt.get_current_fig_manager()
        self.mngr.window.setGeometry(1100, 0, 500, 550)

        self.update_plot() # plot
        
        self.canvas = self.fig.canvas
        self.canvas.mpl_connect('button_press_event', self.button_press_callback)
        self.canvas.mpl_connect('motion_notify_event', self.motion_notify_callback)
        self.canvas.mpl_connect('button_release_event', self.button_release_callback)
        
    def set_sonification(self, init_fn, update_fn, quit_fn):
        self.son_init = init_fn
        self.son_update = update_fn
        self.son_quit = quit_fn

    def update_plot(self):
        if self.plot_flag:
            self.ax.plot(self.targets[:,0], self.targets[:,1], "bo", ms=2)
        self.data = np.array(self.plist)
        if len(self.data): 
            self.p1.set_data(self.data[:,1], self.data[:,2])
        else:
            self.p1.set_data([], [])    
        plt.axis([0,1,0,1])
        plt.pause(0.001)

    def button_press_callback(self, event):
        if self.count == self.num_targets-1:
            print("all targets completed")
            return
        self.plist = []
        self.trial_start_time = time.time()
        if event.inaxes is None: # outside plot area
            return 
        if event.button != 1: # ignore other than left click 
            return
        if (event.xdata-0.5)**2 + (event.ydata-0.5)**2 < 0.01: # start only if clicked in center
            self.run_flag = True
            if callable(self.son_init):
                self.son_init(self)  # initialize sonification
            self.update_plot()

    def motion_notify_callback(self, event):
        if not self.run_flag:
            return
        if event.inaxes is None: 
            return
        if event.button != 1: 
            return
        self.pos = event.xdata, event.ydata # current position of mouse
        vector = [self.pos[0], self.pos[1], self.target[0], self.target[1]] 
        t = time.time()-self.trial_start_time
        self.plist.append([t] + vector)  # save mouse position history for plotting afterwards
        distance = np.sum((self.pos - self.target)**2)**0.5
        if distance < 0.02: # if target reached set next target
            if self.count < self.num_targets - 1:
                %sc Synth(\s1, [\freq, 1500, \num, 1])
                self.count += 1
                self.target = self.targets[self.count]
            else:
                %sc Synth(\s1, [\freq, 2000, \num, 1])
            self.run_completed()
        else:
            if callable(self.son_update):
                self.son_update(self, self.pos, self.target) 

    def report_results(self):
        res = []
        for i, r in enumerate(self.runs):
            tmax = r[-1, 0]
            nsteps = np.shape(r)[0]
            cum_dist = np.sum(np.linalg.norm(np.diff(r[:, 1:3], axis=0), axis=1))
            nom_dist = np.linalg.norm(r[0, 3:]-[0.5,0.5])
            print("trial {0:5} {1:8.4}s, n={2:0}, cum_dist={3:7.3}, nom_dist={4:5.5}".format(
                i, tmax, nsteps, cum_dist, nom_dist))
            res.append([tmax, nsteps, cum_dist, nom_dist] + list(r[0,3:]))
        return np.array(res)
        
    def run_completed(self):
        if callable(self.son_quit):
            self.son_quit(self)
        self.run_flag = False
        self.update_plot()
        self.runs.append(self.data)
        if self.count == self.num_targets-1: # report_results
            self.report_results()
            
    def button_release_callback(self, event):
        if event.button != 1: 
            return
        if not self.run_flag:
            return
        self.run_completed()

In [7]:
stt = StaticTargetTask(num_targets=5, plot_flag=False)
# note: sonification will only work after adding callbacks with next two cells

In [8]:
%%sc // define your synth
SynthDef("cont", { |out=0, freq=100, f1=800, bw=500, amp=0.1, num=1, pos=0 |
    Out.ar(out, Pan2.ar( Formant.ar(freq, f1, bw), pos, amp));
}).add();
// some test code
s.makeBundle(0.1, {x = Synth.new("cont", ["freq", 100, "f1", 400, "bw", 400])});
s.makeBundle(0.2, {x.free});

In [9]:
def son_init(self):
    sc.msg("/s_new", ["cont", 1050, 0, 0, "freq", 100])

def son_quit(self):
    sc.msg("/n_free", [1050])

def son_update(self, pos, target):
    distance = np.sum((pos - target)**2)**0.5
    pos = -np.sign(pos[0]-target[0])
    # default: sonification depends on distance alone 
    freq = scn.midicps(scn.linlin(distance, 0, 1.5, 84, 50))
    sc.msg("/n_set", [1050, "freq", freq, "pos", pos])

stt.set_sonification(son_init, son_update, son_quit)

trial     0    10.88s, n=794, cum_dist=   4.54, nom_dist=0.58382
trial     1    5.835s, n=417, cum_dist=   1.47, nom_dist=0.48417
trial     2     4.98s, n=309, cum_dist=   2.09, nom_dist=0.42112
trial     3     3.28s, n=191, cum_dist=   2.11, nom_dist=0.46306
all targets completed


# Evaluation of the results

In [ ]:
# print and get results
res = stt.report_results()
# plot trajectories
f2 = plt.figure(2)
ax = f2.subplots(2)
for r in stt.runs:
    # plot trajectories
    ax[0].plot(r[:,1], r[:,2], "-", lw=0.5)
    ax[0].plot(r[0,3], r[0,4], 'bo', ms=4)
    # plot error in x and y over time
    ax[1].plot(r[:,0], r[:,4]-r[:,2],"r-")
    ax[1].plot(r[:,0], r[:,3]-r[:,1],"b-");
ax[0].axis('equal')

In [ ]:
# save runs to pickle file
with open('runs-'+datetime.now().strftime("%Y%m%d%H%M%S")+'.pickle', 'wb') as handle:
    pickle.dump(stt.runs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# load runs for later analysis
fname = 'runs-20180725001442.pickle'
with open(fname, 'rb') as handle:
    runs_loaded = pickle.load(handle)